# Project 2

In this assignment you will use the Openweathermap Current Whether Data API to access weather information for cities in the UK. You need to sign up for an account with Openweathermap to get a Key. You will get 1,000 API calls per day for free. https://openweathermap.org/api

#### Q1. Get a list of all cities in the UK (excluding Crown Dependencies and Overseas Territories).
Hint: To get the name of all cities in UK, consider scraping this website https://www.gov.uk/government/publications/list-of-cities/list-of-cities-html.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.gov.uk/government/publications/list-of-cities/list-of-cities-html"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

uk_cities = []

# Flag to start adding cities in the "United Kingdom" section
uk_section = False

# Loop through all headers (h3, h4) and unordered lists (ul) to identify sections
for header in soup.find_all(['h3', 'h4', 'ul']):
    # Look for the "United Kingdom" section and subsequent sub-sections (England, Northern Ireland, Scotland, Wales)
    if header.name == 'h3' and header.get_text(strip=True) == 'United Kingdom':
        uk_section = True  # Start scraping cities inside the UK section
    elif header.name == 'h4' and header.get_text(strip=True) in ['England', 'Northern Ireland', 'Scotland', 'Wales']:
        # Scrape cities within each of these sub-sections
        uk_section = True
    elif header.name == 'ul' and uk_section:
        # Process city names within the 'ul' tags
        for li in header.find_all('li'):
            city_name = li.get_text(strip=True).split('*')[0]  # Remove any "*" from city names
            uk_cities.append(city_name)
    elif header.name == 'h3' and header.get_text(strip=True) in ['Crown Dependencies', 'Overseas Territories']:
        # Stop scraping once we hit Crown Dependencies or Overseas Territories
        uk_section = False

print(uk_cities)


['Bath', 'Birmingham', 'Bradford', 'Brighton & Hove', 'Bristol', 'Cambridge', 'Canterbury', 'Carlisle', 'Chelmsford', 'Chester', 'Chichester', 'Colchester', 'Coventry', 'Derby', 'Doncaster', 'Durham', 'Ely', 'Exeter', 'Gloucester', 'Hereford', 'Kingston-upon-Hull', 'Lancaster', 'Leeds', 'Leicester', 'Lichfield', 'Lincoln', 'Liverpool', 'London', 'Manchester', 'Milton Keynes', 'Newcastle-upon-Tyne', 'Norwich', 'Nottingham', 'Oxford', 'Peterborough', 'Plymouth', 'Portsmouth', 'Preston', 'Ripon', 'Salford', 'Salisbury', 'Sheffield', 'Southampton', 'Southend-on-Sea', 'St Albans', 'Stoke on Trent', 'Sunderland', 'Truro', 'Wakefield', 'Wells', 'Westminster', 'Winchester', 'Wolverhampton', 'Worcester', 'York', 'Armagh', 'Bangor', 'Belfast', 'Lisburn', 'Londonderry', 'Newry', 'Aberdeen', 'Dundee', 'Dunfermline', 'Edinburgh', 'Glasgow', 'Inverness', 'Perth', 'Stirling', 'Bangor', 'Cardiff', 'Newport', 'St Asaph', 'St Davids', 'Swansea', 'Wrexham']


#### Q2. Use the Openweathermap Current Whether Data API to access the following information for each city: **main.temp, wind.speed, rain (1h) and Time of data calculation (dt)**. Use default units for these variables. Add the city name, temperature, wind, rain and dt information to a DataFrame as separate columns. If one of these attributes is missing, replace it with NONE.
Hint: You may find it easier to create a dictionary with cities as keys and temperature, wind, rain and date as values first and then convert it into a DataFrame.

In [3]:
api_key = "4a50b232f60d0cec98d2ea48601340e9"

weather_data = []

# Get weather data
for city in uk_cities:
    if city == "Bangor":
        continue    # Skip 'Bangor'
    if city == "Stoke on Trent": #Stoke on Trent is simply Stoke on Openweathermap
        query = f"q=Stoke,gb"
    else:
        query = f"q={city},gb"
        
    url = f'http://api.openweathermap.org/data/2.5/weather?{query}&APPID={api_key}'

    # Send GET request to the API
    res = requests.get(url)

    # Process the data if request is successful
    if res.status_code == 200:
        data = res.json()

        # Extract relevant information with default values for missing data
        temp = data['main'].get('temp', 'NONE') 
        wind_speed = data['wind'].get('speed', 'NONE')
        rain = data.get('rain', {}).get('1h', 'NONE')
        dt = data.get('dt', 'NONE')

        # Append the data to the list
        weather_data.append({
            'City': city,
            'Temperature': temp,
            'Wind Speed': wind_speed,
            'Rain (1h)': rain,
            'Data Calculation Time': dt
        })
    else:
        print(f"No data for {city}")

# Convert the list to a DataFrame
weather = pd.DataFrame(weather_data)

print(weather)


               City  Temperature  Wind Speed Rain (1h)  Data Calculation Time
0              Bath       280.85        1.86      NONE             1731099131
1        Birmingham       280.17        2.06      NONE             1731098881
2          Bradford       281.58        2.57      NONE             1731099119
3   Brighton & Hove       280.91        0.45      NONE             1731099131
4           Bristol       281.05        2.06      NONE             1731099006
..              ...          ...         ...       ...                    ...
69          Newport       281.24        3.51      NONE             1731098847
70         St Asaph       280.94        0.45      NONE             1731099135
71        St Davids       282.57        6.19      NONE             1731099135
72          Swansea       282.31        4.47      NONE             1731099135
73          Wrexham       280.39        2.06      NONE             1731098937

[74 rows x 5 columns]


#### Q3. The Time of data calculation (dt) is in Unix Timestamp format. Convert it to standard UTC (``dd/mm/yy @ HH:MM:SS``). Print out the updated DataFrame.
Hint: You may use ``datetime.fromtimestamp`` to convert dt.

In [4]:
from datetime import datetime

# Convert the 'dt' column to standard UTC datetime
weather['Data Calculation Time'] = weather['Data Calculation Time'].apply(
    lambda x: datetime.fromtimestamp(x).strftime('%d/%m/%y @ %H:%M:%S') if x != 'NONE' else 'NONE'
)

print(weather)


               City  Temperature  Wind Speed Rain (1h) Data Calculation Time
0              Bath       280.85        1.86      NONE   08/11/24 @ 20:52:11
1        Birmingham       280.17        2.06      NONE   08/11/24 @ 20:48:01
2          Bradford       281.58        2.57      NONE   08/11/24 @ 20:51:59
3   Brighton & Hove       280.91        0.45      NONE   08/11/24 @ 20:52:11
4           Bristol       281.05        2.06      NONE   08/11/24 @ 20:50:06
..              ...          ...         ...       ...                   ...
69          Newport       281.24        3.51      NONE   08/11/24 @ 20:47:27
70         St Asaph       280.94        0.45      NONE   08/11/24 @ 20:52:15
71        St Davids       282.57        6.19      NONE   08/11/24 @ 20:52:15
72          Swansea       282.31        4.47      NONE   08/11/24 @ 20:52:15
73          Wrexham       280.39        2.06      NONE   08/11/24 @ 20:48:57

[74 rows x 5 columns]
